# STAT40800 Data Programming with Python
## Dr. Áine Byrne


# Week 9

This week we cover statistical modelling in Python. We'll begin with basic statistical tests in scipy.
Before moving on to the statsmodels package to explore a linear regression and a logistic regression.

Unfortunately, the documentation (especially examples) for statistical modelling in Python is limited. Extra reading for this lecture includes the [scipy stats tutorial](docs.scipy.org/doc/scipy/reference/tutorial/stats.html) and the
[statsmodels documentation](statsmodels.sourceforge.net/stable/index.html)



In [ ]:
import numpy as np
import pandas as pd
from pandas import DataFrame, Series
import numpy.random as npr
import matplotlib.pyplot as plt

This week we will use two data sets from Elements of Statistical Learning by Hastie, Tibshirani and Friedman.

* The first is the prostate dataset, which contains information about prostate cancer patients. The response variable is the log of the prostate specific antigen, while the explanatory variables are things like age, weight and cancer size. 
* The second is the South African heart disease dataset. The response variable is a Boolean, indicating whether or not the subject has heart disease. The explanatory variables include age, systolic blood pressure and tobacco consumption.

We will load them in directly from their web address.

In [ ]:
prostate = pd.read_csv('http://statweb.stanford.edu/~tibs/ElemStatLearn/datasets/prostate.data',sep='\t')
SA = pd.read_csv('http://statweb.stanford.edu/~tibs/ElemStatLearn/datasets/SAheart.data')

We'll start by having a look at the prostate dataset.

In [ ]:
prostate.head()

In [ ]:
print(prostate.describe())

Before going any further, we will need to standardise the data.

In [ ]:
prostate_numeric = prostate.drop(['Unnamed: 0','train'],axis=1)
prostate_std = (prostate_numeric-prostate_numeric.mean())/prostate_numeric.std()

In [ ]:
print(prostate_std.corr())

With the data standardised, it is easy to identify outliers.

In [ ]:
print(prostate_std[(np.abs(prostate_std)>3).any(1)])

**Note:** `any(x)` method finds every row where this is at least x Trues.

#### Exercise 1

Standardise the South African heart disease dataset and identify any outliers that are at least 4 standard deviations away from the mean. How many of these outliers are there in the tobacco column?

# Hypothesis Testing

Most advanced stats is shared between the scipy.stats and statsmodels modules. We'll begin by looking at scipy.stats.

In [ ]:
import scipy.stats as stats

## Two-sample t-test

Let's see whether lpsa (log prostate specific antigen - the key
response variable) is significantly different between those aged
65 and under versus those aged over 65.

We first need to split the data into two groups.

In [ ]:
lpsa_gt_65 = prostate.lpsa[prostate.age>65]
lpsa_lt_65 = prostate.lpsa[prostate.age<=65]

Then we can run an independent two-sample t-test, using the scipy.stats function `ttest_ind`. The input arguments are the two samples.

In [ ]:
stats.ttest_ind(lpsa_gt_65, lpsa_lt_65)

This returns a tuple of length 2, with the t-statistic followed by the p-value. The default is to assume equal varaince, for unequal variance inlcude the input argument `equal_var=False`. We should also note that scipy.stats only performs two-sided tests. 

#### Exercise 2
Run a t-test with unequal variances on the lpsa data set to see whether there is a significant difference between the training (`train='T'`) and test (`train='F'`) set. What is the p-value?

## Mann Whitney U test

The t-test assumes that the sample is drawn from a normal distribution. If that is not the case, we must use a
non-parametric hypothesis test. The Mann Whitney U test is a non-parametric test, which makes no assumptions about the distribution of the data being sampled.

To run a Mann Whitney U test use the scipy.stats function `mannwhitneyu`.

In [ ]:
stats.mannwhitneyu(lpsa_gt_65, lpsa_lt_65)

As with the t-test this returns the test statistic value followed by the p-value.

## Kolmogorov-Smirnov

Suppose we want to check the shape of the distribution of lpsa. We should probably start by creating a histogram

In [ ]:
plt.figure()
prostate.lpsa.hist()

We can run a Kolmogorov-Smirnov test to test the distribution shape, using the scipt.stats command `kstest`. The first argument is the data we wish to test and the second is the name of the distribution we wish to test it against. A list of scipy.stats distributions and their names can be found [here](https://docs.scipy.org/doc/scipy/reference/stats.html).

In [ ]:
stats.kstest(prostate.lpsa, 'norm')

This result is significant, i.e. the data is not match a standard normal distribution. However, if we se the standardised data, the result is not significant, indicating that the data is normally distibuted.

In [ ]:
stats.kstest(prostate_std.lpsa, 'norm')

#### Exercise 3
The scipy.stats function `pearsonr` performs a statistical test to determine whether or not two variables are linearly dependent. Read the function help file/documentation and use it to test whether the `lpsa` and `lweight` variables are linearly dependent. What is the p-value? 

# QQ-plots

QQ-plots are useful for comparing data to theoretical distributions. In particular, they are great for checking the tails of the distribution. The idea is to plot the data against theoretical quantiles for a
distribution. If the data matches the distribution we should have a straight line.

To create a QQ-plot with scipy.stats, use the `probplot` command. By default the function only returns the values and does not plot them. To create a plot, include the argument `plot=plt`.

In [ ]:
plt.figure()
stats.probplot(prostate.lpsa, dist='norm',plot=plt);

# Linear Regression

The statsmodels package runs lots of different statistical
modelling techniques: linear regression, generalised linear
models, ANOVA, time series models, etc

We'll start with linear regression. Structure:
$$ y = X\beta + \epsilon\hspace{0.1cm}; \hspace{0.5cm}\epsilon \sim N(0,\sigma^2) $$

Typically, we use least squares or maximum likelihood to find the $\beta$ that fits the data best.


In the statsmodels documentation X are called the exogenous variables and y the
endogenous variable.

The original way to fit a linear regression in statsmodels is via
the `OLS` function from the api module.

In [ ]:
import statsmodels.api as sm
mod = sm.OLS(prostate_std.lpsa,prostate_std.drop('lpsa',axis=1))
res = mod.fit()
print(res.summary())

This gives lots of fitting details, but notice it contains no intercept term! If using the api `OLS` function, you need to add an intercept column to the exogenous/explanatory variables.

In [ ]:
X_with_const = prostate_std.drop('lpsa',axis=1)
X_with_const.insert(0,'intercept',1)

In [ ]:
mod = sm.OLS(prostate_std.lpsa,X_with_const)
res = mod.fit()
print(res.summary())

For anyone who has used R before, you may be more comfortable with the other method for fitting a linear regression model using statsmodels. This is done via the `ols` function from the formula.api module.

In [ ]:
import statsmodels.formula.api as smf
mod2 = smf.ols(formula='lpsa ~ lcavol + lweight + age', data=prostate_std)
res2 = mod2.fit()
print(res2.summary())

This method includes an intercept by default. You should read through the help files for `ols` and `fit` to see all of the available input arguments. Additional arguments include `method` which allows you change the fitting method and `subset` which assigns the subset of data to be used in the model it.

In general, use method 2 where you have small numbers of
explanatory (exogenous) variables, and method 1 where you
have large numbers. In addition, method 2 allows you to easily include interaction terms:

In [ ]:
mod3 = smf.ols(formula='lpsa ~ lcavol* lweight * age', data=prostate_std)
res3 = mod3.fit()
print(res3.summary())

You can also include categorical variables.

In [ ]:
prostate_std['age_lt_65'] = prostate.age<65
mod4 = smf.ols(formula='lpsa ~ lcavol + lweight + C(age_lt_65)', data=prostate_std).fit()
print(mod4.summary())

While adding a `-1` to you formula will remove the intercept.

In [ ]:
mod5 = smf.ols(formula='lpsa ~ lcavol + lweight + C(age_lt_65) -1', data=prostate_std).fit()
print(mod5.summary())

Finally, you can add functions of the variable to the formula.

In [ ]:
mod6 = smf.ols(formula='lpsa ~ lcavol + pow(lcavol,2)', data=prostate_std).fit()
print(mod6.summary())

There are a number of attributes and methods that can be applied to the model fit (`mod6` in last example). You can type the name of the model fit followed by a fullstop and hit the tab key, or use the `dir` function to see all of the available methods and attributes.

In [ ]:
mod6.

In [ ]:
dir(mod6)

A particularly useful attribure is `resid` which returns the residuals of the model fit. One assumption of a linear regression model is that the variance of the residual is the same for all values of X. We can verify this by plotting the model prediction against the residuals.

In [ ]:
mod_summary = DataFrame({'preds':mod6.predict(),'resids':mod6.resid})
mod_summary.plot('preds','resids',kind='scatter')

#### Exercise 4
The code below splits the prostate dataset into training and test sets, and fits a linear regression model to the training set only. 

Use the `predict` method to predict the `lpsa` for the test set, using the model fit for the training data. Plot the predictions against the true `lpsa` and compute the correlation between them. Is the model a good fit?


In [ ]:
train_data = prostate_std[prostate.train=='T']
test_data = prostate_std[prostate.train=='F']
train_model = smf.ols(formula='lpsa ~ lcavol + pow(lcavol,2)', data=train_data).fit()

# Logistic regression

Logistic regression is exactly the same as standard linear
regression except that the response variable (endogenous
variable) is binary and treated as Bernoulli distributed variable
$$y_i \sim \text{Bernoulli}(p_i)\hspace{0.1cm}; \hspace{0.5cm}\text{logit}(p_i) = X\beta$$
where $\text{logit}(z) = log(z/(1-z))$. The model is usually fit via maximum
likelihood.

![title](linear-regression-vs-logistic-regression.png)

Taken from: https://static.javatpoint.com/tutorial/machine-learning/images/linear-regression-vs-logistic-regression.png

For this section we will use the SA heart disease dataset. We'll have a quick look at the data and standardise it.

In [ ]:
SA.head()

In [ ]:
SA.describe()

In [ ]:
SA_numeric = SA.drop(['row.names','famhist','chd'],axis=1)
SA_std = (SA_numeric-SA_numeric.mean())/SA_numeric.std()
SA_std.insert(0,'intercept',1)

You should do some plotting and other exploratory data analysis steps here. 

To perform a logistic regression model, we use the `Logit` function from the api module. The syntax is the same as for the `OLS` function.

In [ ]:
logit1 = sm.Logit(SA.chd, SA_std).fit()
print(logit1.summary())

Initial values suggest age, tobacco, ldl cholesterol, and type a behaviour are the most important variables. Remember that these coefficients are given on the logit scale. 
As before loads of other methods and attributes can be found using `dir(logit1)` or `logit1.` and the tab key.

Age seems to be the most important variable, so we will re-fit with age only and then make a prediction for a range of new values. 

In [ ]:
logit2 = sm.Logit(SA.chd, SA_std['age']).fit()
new_age = DataFrame({'age': np.linspace(-3,3,100)})
new_preds = logit2.predict(new_age)
new_age['preds'] = new_preds

In [ ]:
plt.figure()
new_age.plot('age','preds')
plt.plot(SA_std['age'],SA.chd,'r+')
plt.ylim(-0.05,1.05)

Not a strong relationship, despite being the most significant variable.

The model predictions the probability of a person with a particular age (in terms of standard deviations away from the mean) having heart disease. To classify the success of our model fit, we set a cut-off value as with the Week 8 assessed exercises. For a logistic regression model, the cut-off is typically set at 0.5. We will look at this in more detail next week.

We can also use the smf formula interface to fit a logistic regression model. The code below performs a logistic regression using the variables `sbp` and `tobacco`.

In [ ]:
logit_smf = smf.logit(formula='chd ~ sbp + tobacco + famhist', data=SA).fit()
print(logit_smf.summary())

#### Exercise 5
Fit each of the models below to the SA heart disease data. Which relationship results in the best fit (has the highest Pseudo-R$^2$ value)?
* 'chd ~ adiposity + tobacco + ldl'
* 'chd ~ sbp + tobacco + famhist'
* 'chd ~ adiposity + tobacco'
* 'chd ~ tobacco + ldl'